In [135]:
import praw
import datetime as dt
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
from urllib.parse import urljoin
import pandas as pd
from bs4 import BeautifulSoup
import time
import logging

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

This is WebScraper for Reddit

In [136]:
## This is pulling my credentials from a secret config.py file which will not be shared
from config import client_id, client_secret, user_agent, password, username

# Set up PRAW with your credentials
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    password=password,
    user_agent=user_agent,
    username=username,
)

In [ ]:
subreddits = ['N_N_N', 'CombatFootage', 'israelpalestine', 'AskMiddleEast', 'israelhamaswar']
keywords = ['palestine', 'israel', 'gaza', 'west bank', 'hamas', 'idf', 'settlements']

def is_relevant(text):
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in keywords)

def get_top_posts(subreddit_name):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []
    for post in subreddit.top('day', limit=30):  # Fetches top 30 posts of the day
        if is_relevant(post.title) or is_relevant(post.selftext): ## Searches for the relevent keywords in the title and text
            posts.append(post)
    return posts[:15]  # Limit to 15 relevent posts

def get_comments(post):
    post.comments.replace_more(limit=0)  # This line ensures that all MoreComments objects are removed
    comments = post.comments.list()
    return comments[:5]  # Limit to the first 5 comments

def scrape_data():
    new_data = []
    for subreddit_name in subreddits:
        top_posts = get_top_posts(subreddit_name)
        for post in top_posts:
            post_data = {
                'title': post.title,
                'text': post.selftext,
                'upvotes': post.score,
                'author': str(post.author),
                'date': dt.datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'id': post.id,
                'flair': post.link_flair_text,
                'type': 'post',
                'parent_id': None,
                'subreddit': subreddit_name
            }
            new_data.append(post_data)
            comments = get_comments(post)
            for comment in comments:
                comment_data = {
                    'title': None,
                    'text': comment.body,
                    'upvotes': comment.score,
                    'author': str(comment.author),
                    'date': dt.datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                    'id': comment.id,
                    'flair': None,
                    'type': 'comment',
                    'parent_id': comment.parent_id,
                    'subreddit': subreddit_name
                }
                new_data.append(comment_data)

    return new_data

# Load existing data if available
try:
    existing_data = pd.read_csv('reddit_data_combined.csv')
except FileNotFoundError:
    existing_data = pd.DataFrame()

# Function to run the data scraping and saving
def run_data_collection():
    print(f"Scraping data for {dt.datetime.now().strftime('%Y-%m-%d')}")
    daily_data = scrape_data()
    df_daily = pd.DataFrame(daily_data)

    # Combine with existing data and remove duplicates
    combined_data = pd.concat([existing_data, df_daily]).drop_duplicates(subset='id').reset_index(drop=True)

    # Save the combined data
    combined_data.to_csv('reddit_data_combined.csv', index=False)
    print(f"Data scraped and saved to reddit_data_combined.csv")

# Run the data collection function directly for now
run_data_collection()

WEBSCRAPER 2: AL JAZEERA:
This is for https://www.aljazeera.com/tag/israel-palestine-conflict/

In [143]:
# Function to set up logging
def setup_logging():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to scrape the body of an article and the publication date
def scrape_article_body_and_date(driver):
    try:
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#main-content-area p"))
        )
        html_source = driver.page_source
        article_soup = BeautifulSoup(html_source, 'html.parser')

        # Extracting the article body
        content_container = article_soup.select_one("#main-content-area")
        paragraphs = content_container.find_all('p')
        article_body = ' '.join(paragraph.get_text(strip=True) for paragraph in paragraphs)

        # Extracting the article date
        date_element = article_soup.select_one(".article-dates .date-simple span")
        article_date = date_element.get_text(strip=True) if date_element else 'Date not found'

        return article_body, article_date
    except TimeoutException:
        logging.warning("Timed out waiting for page to load")
        return 'Content loading timed out', ''
    except Exception as e:
        logging.error(f"An error occurred while scraping the article body: {e}")
        return 'Error while scraping', ''

def get_article_urls_and_titles(driver, visited_urls):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    new_articles = []
    articles = soup.find_all('article')
    logging.info(f"Found {len(articles)} articles on the page.")
    for article in articles:
        title_element = article.find('h3')
        link_element = article.find('a', href=True)
        if title_element and link_element:
            title = title_element.get_text(strip=True)
            url = urljoin(base_url, link_element['href'])
            if url not in visited_urls:
                visited_urls.add(url)
                new_articles.append({'title': title, 'url': url})
    return new_articles

def click_show_more(driver):
    try:
        show_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.show-more-button"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
        show_more_button.click()
        return True
    except TimeoutException:
        logging.info("No 'Show more' button to click or timed out waiting for the button to be clickable.")
        return False

setup_logging()

# Set up Chrome options to run the browser visibly (not headless)
options = Options()
options.headless = False  # Disable headless mode to visualize the browser actions
options.add_argument("--window-size=1920,1200")

# Instantiate a new driver
driver = webdriver.Chrome(options=options)

base_url = 'https://www.aljazeera.com'
driver.get(base_url + '/tag/israel-palestine-conflict/')

# Wait for the initial articles to load
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "article")))

# Attempt to close any overlaying elements like cookie consent banners
try:
    close_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.close, div.cookie-consent button, a[href*='cookie']"))
    )
    close_button.click()
except (TimeoutException, NoSuchElementException, ElementClickInterceptedException):
    logging.info("No overlay found or not clickable.")

# Initialize a set for visited URLs
visited_urls = set()

# Load existing data
csv_file = 'data_aljazeera_withdates.csv'
try:
    existing_data = pd.read_csv(csv_file)
    existing_urls = set(existing_data['url'])
    logging.info("Loaded existing data.")
except FileNotFoundError:
    existing_data = pd.DataFrame()
    existing_urls = set()
    logging.info("No existing file found. Creating a new dataset.")

# Scrape the initial set of articles
all_articles = get_article_urls_and_titles(driver, visited_urls)

# Click the 'Show more' button to reveal more articles
for _ in range(30):
    if not click_show_more(driver):
        break
    time.sleep(5)  # Wait for the page to load more articles
    all_articles.extend(get_article_urls_and_titles(driver, visited_urls))

# Now visit each URL and get the body content and date
new_articles = []
for article in all_articles:
    if article['url'] not in existing_urls:
        try:
            driver.get(article['url'])
            time.sleep(2)  # Ensure the page has loaded
            article_body, article_date = scrape_article_body_and_date(driver)
            article['body'] = article_body
            article['date'] = article_date
            new_articles.append(article)
        except TimeoutException:
            logging.warning(f"Timed out trying to load {article['url']}")
            continue

# Combine new articles with existing data
if new_articles:
    df_new_articles = pd.DataFrame(new_articles)
    combined_data = pd.concat([existing_data, df_new_articles], ignore_index=True)
    logging.info("New articles added.")
else:
    combined_data = existing_data
    logging.info("No new articles to add.")

# Save the combined data to a CSV file
combined_data.to_csv(csv_file, index=False)
logging.info(f"Data saved to {csv_file}.")

# Close the driver
driver.quit()

2023-12-08 21:12:36,858 - INFO - No overlay found or not clickable.
2023-12-08 21:12:37,036 - INFO - Loaded existing data.
2023-12-08 21:12:37,131 - INFO - Found 14 articles on the page.
2023-12-08 21:12:42,295 - INFO - Found 24 articles on the page.
2023-12-08 21:12:47,451 - INFO - Found 34 articles on the page.
2023-12-08 21:12:53,643 - INFO - Found 44 articles on the page.
2023-12-08 21:12:58,824 - INFO - Found 54 articles on the page.
2023-12-08 21:13:03,998 - INFO - Found 64 articles on the page.
2023-12-08 21:13:09,193 - INFO - Found 74 articles on the page.
2023-12-08 21:13:14,391 - INFO - Found 84 articles on the page.
2023-12-08 21:13:19,603 - INFO - Found 94 articles on the page.
2023-12-08 21:13:24,837 - INFO - Found 104 articles on the page.
2023-12-08 21:13:30,067 - INFO - Found 114 articles on the page.
2023-12-08 21:13:35,481 - INFO - Found 124 articles on the page.
2023-12-08 21:13:40,734 - INFO - Found 134 articles on the page.
2023-12-08 21:13:45,972 - INFO - Found 14

This is the scraper for news.com.au

In [142]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to scrape the body of an article and the publication date
def scrape_article_body_newscomau(driver):
    try:
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "story-body"))
        )
        html_source = driver.page_source
        article_soup = BeautifulSoup(html_source, 'html.parser')

        # Extracting the article body
        content_container = article_soup.find(id="story-body")
        paragraphs = content_container.find_all('p')
        article_body = ' '.join(paragraph.get_text(strip=True) for paragraph in paragraphs)

        # Extracting the article date
        date_element = article_soup.find('div', class_='byline_publish')
        article_date = date_element.get_text(strip=True) if date_element else 'Date not found'

        return article_body, article_date
    except TimeoutException:
        logging.warning("Timed out waiting for the article's body to load")
        return 'Content loading timed out', ''
    except Exception as e:
        logging.error(f"An error occurred while scraping the article body: {e}")
        return 'Error while scraping', ''

# Define the function to get article URLs and titles
def get_article_urls_and_titles_newscomau(driver, visited_urls):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    new_articles = []
    articles = soup.find_all('article', class_='storyblock')
    logging.info(f"Found {len(articles)} articles on the page.")
    for article in articles:
        title_element = article.find('a', class_='storyblock_title_link')
        if title_element:
            title = title_element.get_text(strip=True)
            url = urljoin('https://www.news.com.au', title_element['href'])
            if url not in visited_urls:
                visited_urls.add(url)
                new_articles.append({'title': title, 'url': url})
    return new_articles

# Set up Chrome options to run the browser visibly (not headless)
options = Options()
options.headless = False
options.add_argument("--window-size=1920,1200")

# Instantiate a new driver
driver = webdriver.Chrome(options=options)

# Initialize a set for visited URLs
visited_urls = set()

base_url = 'https://www.news.com.au/world/middle-east'

# Load existing data if available
csv_file = 'data_newscomau_withdates.csv'
try:
    existing_data = pd.read_csv(csv_file)
    existing_urls = set(existing_data['url'])
    logging.info("Loaded existing data.")
except FileNotFoundError:
    existing_data = pd.DataFrame(columns=['title', 'url', 'body', 'date'])
    existing_urls = set()
    logging.info("No existing file found. Creating a new dataset.")

# Scrape articles from page 1 to page 6
all_articles = []
for i in range(1, 8):  # Page numbers 1 to 6
    page_url = f"{base_url}/page/{i}" if i > 1 else base_url
    driver.get(page_url)
    time.sleep(5)  # Wait for the page to load completely
    all_articles.extend(get_article_urls_and_titles_newscomau(driver, visited_urls))

# Now visit each URL and get the body content and date
new_articles = []
for article in all_articles:
    if article['url'] not in existing_urls:
        driver.get(article['url'])
        time.sleep(2)  # Ensure the page has loaded
        article_body, article_date = scrape_article_body_newscomau(driver)
        article['body'] = article_body
        article['date'] = article_date
        new_articles.append(article)

# Combine new articles with existing data
if new_articles:
    df_new_articles = pd.DataFrame(new_articles)
    combined_data = pd.concat([existing_data, df_new_articles], ignore_index=True)
    logging.info("New articles added.")
else:
    combined_data = existing_data
    logging.info("No new articles to add.")

# Save the combined data to a CSV file
combined_data.to_csv(csv_file, index=False)
logging.info(f"Data saved to {csv_file}.")

# Close the driver
driver.quit()

logging.info("Scraping completed.")

2023-12-08 19:21:55,096 - INFO - No existing file found. Creating a new dataset.
2023-12-08 19:23:10,622 - INFO - Found 30 articles on the page.
2023-12-08 19:23:22,545 - INFO - Found 30 articles on the page.
2023-12-08 19:23:34,036 - INFO - Found 30 articles on the page.
2023-12-08 19:23:45,492 - INFO - Found 30 articles on the page.
2023-12-08 19:23:58,863 - INFO - Found 30 articles on the page.
2023-12-08 19:24:11,259 - INFO - Found 30 articles on the page.
2023-12-08 19:24:23,129 - INFO - Found 30 articles on the page.
2023-12-08 19:52:33,566 - INFO - New articles added.
2023-12-08 19:52:33,674 - INFO - Data saved to data_newscomau_withdates.csv.
2023-12-08 19:52:37,846 - INFO - Scraping completed.
